In [ ]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')


# set from Luisa Jan 2021
set_url = '/search/?experiments_in_set.experiment_type.display_title=ChIP-seq&experimentset_type=replicate&lab.display_title=Karen+Adelman%2C+HARVARD&status=pre-release&type=ExperimentSetReplicate'
all_sets = ff_utils.search_metadata(set_url , key=my_auth)

counter = 0
completed = 0
completed_acc = []

run_sets = [i for i in all_sets if "ENCODE_ChIP_Pipeline_1.1.1"  not in i.get('completed_processes', [])]
print(len(all_sets), 'total number of sets')
print(len(all_sets)-len(run_sets), 'sets completed')
print(len(run_sets), 'ready for processing')

In [ ]:
### need to add bucket choice for step1 if fastqs merged
### control and exp might be different buckets
run_wfr = True
add_pc = False
add_tag = False

for a_set in run_sets: 
    print()
    print(a_set['accession'], end = " ")
    counter += 1
    # some feature to extract from each set
    control = ""  # True or False (True if set in scope is control)
    control_set = ""  # None (if no control exp is set), or the control experiment for the one in scope
    target_type = "" # Histone or TF (or None for control)
    paired = "" # single or paired
    organism = ""
    
    # pass attributions to new objects
    attributions = None
    
    replicate_exps = a_set['replicate_exps']
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])

    # get organism, target and control from the first replicate
    f_exp = replicate_exps[0]['replicate_exp']['uuid']
    f_exp_resp = ff_utils.get_metadata(f_exp, key = my_auth)
    control, control_set, target_type, organism = get_chip_info(f_exp_resp, my_auth)
           
    print('ORG:', organism, "CONT:", control, "TARGET:", target_type, "CONT_SET:", control_set)
    if control and target_type:
        print('SHOULD NOT BE CONTROL')
        continue

    if organism not in ['mouse', 'human']:
        print('orgamism not ready', organism)
        continue

    attributions = get_attribution(ff_utils.get_metadata(f_exp_resp['files'][0]['uuid'], key = my_auth))
    
    if not control:
        if not target_type:
            print('set is not control, but missing target type, skipping')
            continue

    ta = []
    taxcor = []
    ta_cnt = []
    
    # check for step1, and start if missing
    step1_status = 'Done'
    control_ready = True

    for an_exp in replicate_exps:
        my_source = ''
        exp_id = an_exp['replicate_exp']['accession']
        exp_resp = ff_utils.get_metadata(exp_id, my_auth)
        
        run_name = exp_resp['accession']
        exp_files, exp_obj, paired = get_chip_files(exp_resp, my_auth)
        print(run_name, len(exp_files), paired, end = ' status: ')
        
        
        # if too many input, merge them
        if len(exp_files) > 2:
            print('need to implement bucket choice ')
            continue
            my_source = 'processed'
            attributions = get_attribution(ff_utils.get_metadata(exp_files[0][0], key = my_auth))
            # exp_files format [[pair1,pair2], [pair1, pair2]]  uuids
            # exp_obj format  [[pair1,pair2], [pair1, pair2]]  accession.fileformat ('4DNFIKW8IQT2.fastq.gz')
            input_list = []
            if paired == 'paired':
                # first add paired end 1s
                input_list.append([i[0] for i in exp_files])
                input_list.append([i[1] for i in exp_files])
            
            elif paired == 'single':
                input_list.append([i[0] for i in exp_files])
                     
            merged_files = []
            step0_status = 'complete'
            iit = 0
            for merge_case in input_list:
                iit += 1
                all_step0s = []
                for an_input in merge_case:
                    step0_result = get_wfr_out(an_input, 'merge-fastq', my_auth, ['v1'])
                    all_step0s.append((step0_result['status'], step0_result.get('fastq')))
                if len(list(set(all_step0s))) != 1:
                    print('inconsistent step0 run for input fastq files')
                    # this run will be repeated if add_wfr
                    step0_result['status'] = 'inconsistent run'
                #check if part 0 is run already, it not start the run
                # if successful
                if step0_result['status'] == 'complete':
                    merged_fastq = step0_result['fastq']
                    merged_files.append(merged_fastq)
                # if still running
                elif step0_result['status'] == 'running':
                    step0_status = 'running'
                # if run is not successful
                else:
                    step0_status = 'missing'
                    if run_wfr:
                        # RUN PART 0
                        print('\nstarting step0')
                        inp_f = {'input_fastqs':merge_case}
                        tag = exp_id + '_p' + str(iit) 
                        run_missing_wfr(step_settings('merge-fastq', organism, attributions),
                                        inp_f, tag, my_auth, my_env)

            if step0_status != 'complete':
                print('step0', step0_status)
                step1_status = 'not ready'
                continue
            
            # update exp_files and exp_obj
            exp_files = [[]]
            exp_obj = [[]]
            for a_merged in merged_files:
                temp_resp = ff_utils.get_metadata(a_merged, my_auth)
                exp_files[0].append(temp_resp['uuid'])
                exp_obj[0].append(temp_resp['display_title'])
   
        if control:
            step1_result = get_wfr_out_file(exp_files[0][0], 'encode-chipseq-aln-ctl', my_auth, ['1.1.1'], run=10000)
            print('step1', step1_result['status'])
            if step1_result['status'] == 'complete':
                if add_pc:
                    add_preliminary_processed_files(exp_id, [step1_result['chip.first_ta_ctl']], my_auth, run_type = 'chip')
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'                
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-ctl', organism, attributions), 
                                      organism, 'tf', paired, [exp_files], [exp_obj], my_env, my_auth, run_name)
                # print(adana)
        else:
            # check for the status on each experiment
            step1_result = get_wfr_out_file(exp_files[0][0], 'encode-chipseq-aln-chip', my_auth, ['1.1.1']) 
            print('step1', step1_result['status'])
            if step1_result['status'] == 'complete':
                ta.append(step1_result['chip.first_ta'])
                taxcor.append(step1_result['chip.first_ta_xcor'])
                if add_pc:
                    add_preliminary_processed_files(exp_id, [step1_result['chip.first_ta']], my_auth, run_type = 'chip')
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-chip', organism, attributions), 
                                      organism, target_type, paired, [exp_files], [exp_obj], my_env, my_auth, run_name)
                   
            if control_set:
                try:
                    exp_cnt_ids = [i['experiment'] for i in exp_resp['experiment_relation'] if i['relationship_type'] == 'controlled by']
                    exp_cnt_ids = [i['@id'].split('/')[2] for i in exp_cnt_ids]
                except:
                    control_ready = False
                    print('Control Relation has problems for this exp', exp_resp['accession'])
                    continue
                if len(exp_cnt_ids)!= 1:
                    control_ready = False
                    print('Multiple controls for this exp', exp_resp['accession'])
                    continue
                    
                exp_cnt_id = exp_cnt_ids[0]
                print('controled by set', exp_cnt_id)
                exp_cnt_resp = ff_utils.get_metadata(exp_cnt_id, my_auth)
                cont_file = ''
                # check opf for control file               
                for opf_case in exp_cnt_resp.get('other_processed_files', []):
                    if opf_case['title'] == 'ENCODE ChIP-Seq Pipeline - Preliminary Files':
                        opf_files = opf_case['files']
                        assert len(opf_files) == 1
                        cont_file = opf_files[0]['@id']
                # if not in opf, check processed files
                if not cont_file:
                    pf_list = exp_cnt_resp.get('processed_files', [])
                    if pf_list:
                        if pf_list:
                            assert len(pf_list) == 1
                            cont_file = pf_list[0]['@id']

                
                if cont_file:
                    ta_cnt.append(cont_file)
                else:
                    control_ready = False
    print('control set 2', control_set)
    if step1_status != 'Done':
        print('Step1 not complete, skip step2:', step1_status) 
        continue
        
    # add pc files and tag for control
    if control:
        #add competed flag to set
        if add_tag:
            ff_utils.patch_metadata({"completed_processes":["ENCODE_ChIP_Pipeline_1.1.1"]}, obj_id=a_set['accession'] , key=my_auth)
        # add processed files to set
        continue

    if not control_ready:
        print('Control not ready for second step')
        continue
    
    print('set ready for part2')
    print('ta length', len(ta))
    if len(ta) > 3:
        print('skipping more then 3 exp set')
        continue

    if len(ta) > 2:     
        # only covers 3 technical or 3 biological replicates
        ta_2 = []
        taxcor_2 = []        
        print('ExperimentSet has 3 experiments, selecting best 2')
        ta_2 = select_best_2(ta, my_auth)
        # xcor does not have qc, use ta indexes to find the correct files
        for ta_f in ta_2:
            taxcor_2.append(taxcor[ta.index(ta_f)])
        ta = ta_2
        taxcor = taxcor_2
        ta_cnt = select_best_2(ta_cnt, my_auth)

    if control_set:  
        if len(ta_cnt) != len(ta):
            print('Control and experiment have diferent number of bio reps, skipping')
            continue
    
    step2_result = get_wfr_out_file(ta[0], 'encode-chipseq-postaln', my_auth, ['1.1.1'])
    print(ta)
    print(ta_cnt)
    print(step2_result)
    if step2_result['status'] == 'complete':
        print('step2 is complete')
        print('https://data.4dnucleome.org/' + step2_result['chip.sig_fc'])
        if add_pc:
            add_preliminary_processed_files(a_set['accession'], 
                                            [
                                                step2_result['chip.optimal_peak'],
                                                step2_result['chip.conservative_peak'],
                                                step2_result['chip.sig_fc']
                                            ], 
                                            my_auth, run_type = 'chip')
        if add_tag:
            ff_utils.patch_metadata({"completed_processes":["ENCODE_ChIP_Pipeline_1.1.1"]}, obj_id=a_set['accession'] , key=my_auth)   
    elif step2_result['status'] == 'running':
        print('step2 is still running')
    else:
        step2_status = 'Incomplete'  
        print('missing step2')
        if run_wfr:
            print('starting run')
            run_ids = {'run_name': a_set['accession'],
                       "desc" : a_set.get('description', ''),
                      }
            run_missing_chip2(control_set, step_settings('encode-chipseq-postaln', organism, attributions), 
                              organism, target_type, paired, ta, taxcor, ta_cnt, my_env, my_auth, run_ids)



In [ ]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

action = False
move_title = 'ENCODE ChIP-Seq Pipeline - Preliminary Files'

# set_url = '/search/?lab.display_title=Gerd+Blobel%2C+CHOP&type=ExperimentSetReplicate&processed_files.uuid=No+value'
# all_sets = ff_utils.search_metadata(set_url , key=my_auth)
print(len(all_sets))
ready_sets_1 = [i for i in all_sets if "ENCODE_ChIP_Pipeline_1.1.1" in i.get('completed_processes', [])]
print(len(ready_sets_1))
ready_sets_2 = []
for a_set in ready_sets_1:
    if a_set.get('other_processed_files'):
        print(a_set['accession'])
        if move_title in [i['title'] for i in a_set['other_processed_files']]:
            if a_set.get('processed_files'):
                print('WARN' ,a_set['accession'], 'has items in processed files, skipping ')
                continue
            else:
                ready_sets_2.append(a_set)
print(len(ready_sets_2), 'items are ready')

In [ ]:
# move other processed files to processed files field
action = True
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        if opc:
            print('There are files in processed_files field, expected empty', resp['accession'])
            return False
        else:
            print('it is possible that move already happened, no opc but pc', resp['accession'])
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                # release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'ENCODE ChIP-Seq Pipeline - Preliminary Files'

print(len(ready_sets_1), 'experiment sets in scope')
for a_set in ready_sets_1:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_auth, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_auth)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_auth, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_auth)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)

In [ ]:
# disable wfr for control files
q = '/search/?type=FileProcessed&workflow_run_outputs.workflow.title=Chip-seq+data+processing+pipeline%2C+Align-only%2C+Control+1.1.1&disable_wfr_inputs!=true'
control_files = ff_utils.search_metadata(q, my_auth)
print(len(control_files))
# for con_f in control_files:
#     ff_utils.patch_metadata({'disable_wfr_inputs': True}, con_f['uuid'], my_auth)
#     print('+', end='')